In [5]:
!unzip Archive.zip

Archive:  Archive.zip
  inflating: chord_recognition.py    
  inflating: __MACOSX/._chord_recognition.py  
   creating: MuMu_dataset/
  inflating: __MACOSX/._MuMu_dataset  
  inflating: MuMu_dataset/amazon_reviews_MuMu.json  
  inflating: __MACOSX/MuMu_dataset/._amazon_reviews_MuMu.json  
  inflating: MuMu_dataset/amazon_metadata_MuMu.json  
  inflating: __MACOSX/MuMu_dataset/._amazon_metadata_MuMu.json  
  inflating: MuMu_dataset/MuMu_dataset_multi-label.csv  
  inflating: __MACOSX/MuMu_dataset/._MuMu_dataset_multi-label.csv  
  inflating: MuMu_dataset/MuMu_dataset_single-label.csv  
  inflating: __MACOSX/MuMu_dataset/._MuMu_dataset_single-label.csv  
   creating: remi/
  inflating: __MACOSX/._remi         
  inflating: remi/TRTVBXQ128F14750C0.npz  
  inflating: __MACOSX/remi/._TRTVBXQ128F14750C0.npz  
  inflating: remi/TRSZIBQ128F9310F92.npz  
  inflating: __MACOSX/remi/._TRSZIBQ128F9310F92.npz  
  inflating: remi/TRMPDJM128F930D4D8.npz  
  inflating: __MACOSX/remi/._TRMPDJM128F930D4

In [6]:
% pip install miditoolkit

     |████████████████████████████████| 51 kB 4.1 MB/s 


In [7]:
import glob
import os
import json
import csv
import random
import utils
import numpy as np
import pandas as pd


In [8]:
MATCHED_PATH = "lpd_5/lpd_5_matched/"
CLEANED_PATH = "lpd_5/lpd_5_cleansed/"
MUMU_AMAZON_PATH = "MuMu_dataset/amazon_reviews_MuMu.json"
MUMU_MULTI_PATH = "MuMu_dataset/MuMu_dataset_multi-label.csv"
MUMU_SINGLE_PATH = "MuMu_dataset/MuMu_dataset_single-label.csv"
REMI_PATH = "remi/"

matched_file_paths = []
for x in os.walk(MATCHED_PATH):
    for y in glob.glob(os.path.join(x[0], '*.npz')):
        matched_file_paths.append(y)

cleaned_file_paths = []
for x in os.walk(CLEANED_PATH):
    for y in glob.glob(os.path.join(x[0], '*.npz')):
        cleaned_file_paths.append(y)

broken_remi = ['TRRPLUG128F427E59C',
 'TRFINAP128F93269EF',
 'TRMSNKX128F9355DDA',
 'TRDRFXO128F14918D8',
 'TRVMEZF128F92C82F6',
 'TRKIZWL128EF342C5A',
 'TRLWVSP128F423C35A']
remi_file_paths = []
for x in os.walk(REMI_PATH):
    for y in glob.glob(os.path.join(x[0], '*.npz')):
        remi_file_paths.append(y)

In [6]:
# total number of songs
print(len(matched_file_paths), len(cleaned_file_paths), len(remi_file_paths))

0 0 0


In [9]:
mumu_multi_df = pd.read_csv(MUMU_MULTI_PATH)
mumu_single_df = pd.read_csv(MUMU_SINGLE_PATH)
mumu_amazon_df = pd.read_json(MUMU_AMAZON_PATH, lines=True)

In [ ]:
#list of MSD and recording ids of the piano roll dataset
cleaned_MSD_id_list = []
uncleaned_MSD_id_list = []
recording_mbid_list = []
for i in range(len(cleaned_file_paths)):
    curr_path = cleaned_file_paths[i]
    curr_MSD_id = curr_path.split('/')[-2]
    cleaned_MSD_id_list.append(curr_MSD_id)
for i in range(len(matched_file_paths)):
    curr_path = matched_file_paths[i]
    curr_MSD_id = curr_path.split('/')[-2]
    uncleaned_MSD_id_list.append(curr_MSD_id)
    recording_mbid_list.append(curr_path.split('/')[-1].split('.')[0])
unique_uncleaned_MSD_id_list = list(set(uncleaned_MSD_id_list))

In [19]:
label_list = []
tuple_list = []
DATA_LENGTH = 20000
NUM_FILE = 100
min_remi_length = 1818
max_remi_length = 0
avg_remi_length = 0

CLEANED = False

idx = 0
while len(tuple_list) < len(remi_file_paths):
#     MSD_ids = cleaned_MSD_id_list if CLEANED else unique_uncleaned_MSD_id_list
#     curr_path = cleaned_file_paths[idx] if CLEANED else matched_file_paths[idx]
#     curr_MSD_id = curr_path.split('/')[-2]
    curr_path = remi_file_paths[idx]
    curr_MSD_id = curr_path.split('.')[0].split('/')[-1]
    idx += 1
    
    curr_amazon_id = mumu_multi_df.loc[mumu_multi_df['MSD_track_id'] == curr_MSD_id]
    if len(curr_amazon_id) == 0:
        curr_amazon_id = mumu_single_df.loc[mumu_single_df['MSD_track_id'] == curr_MSD_id]
        # if len(curr_amazon_id) == 0:
        #     continue
            
    # npz = np.load(curr_path)
    # df= pd.DataFrame.from_dict({item: npz[item] for item in npz.files}, orient='index').T
#     data_nparr = df[['pianoroll_0_csc_data', 'pianoroll_1_csc_data', 'pianoroll_2_csc_data', 'pianoroll_3_csc_data', 'pianoroll_4_csc_data']].fillna(0).to_numpy()
#     if data_nparr.shape[0] > DATA_LENGTH:
#         data_nparr = data_nparr[:DATA_LENGTH + 1]
#     elif data_nparr.shape[0] < DATA_LENGTH:
#         np.pad(data_nparr, (0, DATA_LENGTH - data_nparr.shape[0]), 'constant', constant_values=(0, 0))

    npz = np.load(curr_path, allow_pickle=True)
    data_nparr=npz['arr_0']
    min_remi_length = min(min_remi_length, len(data_nparr))
    max_remi_length = max(max_remi_length, len(data_nparr))
    avg_remi_length += len(data_nparr)

    curr_review = mumu_amazon_df.loc[mumu_amazon_df['amazon_id'] == curr_amazon_id['amazon_id'].values[0]]['reviewText']
    curr_tuple = [data_nparr.tolist(), curr_review.values[0]]
    tuple_list.append(curr_tuple)
    label_list.append(curr_review.values[0])

avg_remi_length /= len(remi_file_paths)

In [20]:
print(min_remi_length, max_remi_length, avg_remi_length)

8 47357 4029.748065348237


In [21]:
neg_tuple_list = []
for tuples in tuple_list:
    neg_label = tuples[1]
    while neg_label == tuples[1]:
        neg_label = random.choice(label_list)
    neg_tuple = [tuples[0], neg_label]
    neg_tuple_list.append(neg_tuple)

In [22]:
POS_TUPLE_PATH = "pos_tuples_mini.csv"
NEG_TUPLE_PATH = "neg_tuples_mini.csv"
fields = ['data', 'label'] 
with open(POS_TUPLE_PATH, 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(tuple_list)
with open(NEG_TUPLE_PATH, 'w') as f:
    write = csv.writer(f)
    write.writerow(fields)
    write.writerows(neg_tuple_list)

In [35]:
tuple_test = []
for i in range(5):
  sample_data = tuple_list[i][0][:2]
  sample_data.append("...")
  sample_label = tuple_list[i][1][:80]
  sample_label += "..."
  tuple_test.append((sample_data, sample_label))
df_test = pd.DataFrame(tuple_test, columns=['data', 'label'])
df_test

,data,label
0,"[Event(name=Bar, time=None, value=None, text=1...",Beware of this album if you are expecting the ...
1,"[Event(name=Bar, time=None, value=None, text=1...",Excellent CD Brings back old memories. Exactly...
2,"[Event(name=Bar, time=None, value=None, text=1...",James TaliaStories To Be Told2014Talking with ...
3,"[Event(name=Bar, time=None, value=None, text=1...",First of all she sounds terrible in almost all...
4,"[Event(name=Bar, time=None, value=None, text=1...",1. 3:50 Kim Carnes - Bette Davis Eyes2. 3:41 L...


In [37]:
min_token_length = 10000
max_token_length = 0
avg_token_length = 0
for label in label_list:
  token_list = label.split(' ')
  min_token_length = min(min_token_length, len(token_list))
  max_token_length = max(max_token_length, len(token_list))
  avg_token_length += len(token_list)
avg_token_length /= len(label_list)

In [38]:
print(min_token_length, max_token_length, avg_token_length)

1 1774 120.20765262252795


In [ ]:
#mintime: 1824
#maxtime: 166926
#avgtime of first 100: 31551.36

#number of cleaned pianoroll tracks: 21425
#cleaned tracks matched to unique labels: 2504 / 1916 / 1701 / 1816 /2333
#all matching labels of cleaned tracks: 39776 / 40172

#number of uncleaned pianoroll recordings: 115160
#unique uncleaned tracks matched to unique labels: 3559 / 3328
#all matching labels of unique uncleaned tracks: 76861
#all uncleaned recordings matched to repetative lablels: 11697

#size of entire mumudataset: 956292
# There may be multiple reviews to the same MSD track!!!

#count pianoroll matched to amazon review
count = 0
count_review = 0
matchable_ids = []
for idd in cleaned_MSD_id_list:
    curr_amazon_id = mumu_single_df.loc[mumu_single_df['MSD_track_id'] == idd]
    if len(curr_amazon_id) == 0:
        curr_amazon_id = mumu_multi_df.loc[mumu_multi_df['MSD_track_id'] == idd]
        if len(curr_amazon_id) == 0:
            continue
    matchable_ids.append(idd)
    curr_review = mumu_amazon_df.loc[mumu_amazon_df['amazon_id'] == curr_amazon_id['amazon_id'].values[0]]['reviewText']
    count += 1
    count_review += len(curr_review.values)

In [ ]:
textfile = open("matchable_ids.txt", "w")
for matchable_id in matchable_ids:
    textfile.write(matchable_id + "\n")
textfile.close()

In [ ]:
print(len(matchable_ids))

2333
